<a href="https://colab.research.google.com/github/azizamirsaidova/abstractive_text_summarization/blob/main/Fine_tune_simplet5_absractive_text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rouge
!pip install tf-estimator-nightly==2.8.0.dev2021122109
!pip install folium==0.2.1
!pip install simplet5 -q

from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
import pandas as pd
import nltk
from random import sample
import torch
torch.cuda.empty_cache()
import os
import torch
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from random import sample
from rouge import Rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79793 sha256=b3c9f0c9e23ab574e1b2f9c06f477402485d639d18fd6fc0124061206ce63958
  Stored in directory: /root/.cache/pip/wheels/00/0c/07/d7792a5444d5bb074361ac27da53cee9d5cce59a07fe9da5dd
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.14.0
    Uninstalling folium-0.14.0:
      Successfully uninstalled folium-0.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.17.6 requires folium>=0.9.1, but you have folium 0.2.1 which is incompatible.
geemap 0.28.2 requires folium>=0.13.0, but you have folium 0.2.1 which is incompatible.
  Pr

In [2]:
path = '/content/gdrive/MyDrive/abstractive_text_summarization-main/data/train.csv'
trainning_df = pd.read_csv(path, engine='python', error_bad_lines=False)

MAX_LEN = 512
SUMMARY_LEN = 150
TRAINNING_SIZE = 5000

trainning_df = trainning_df.iloc[0:TRAINNING_SIZE,:].copy()
trainning_article_ls = list(trainning_df['article'])
trainning_highlight_ls = list(trainning_df['highlights'])


<ipython-input-2-4a02521e8e1f>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  trainning_df = pd.read_csv(path, engine='python', error_bad_lines=False)


In [3]:
for index in sample(list(np.arange(len(trainning_article_ls))),3):
    print('Original Text : ')
    print(trainning_article_ls[index])

    print('\n\nSummary Text : ')
    print(trainning_highlight_ls[index])

    print('===========================================================================================================\n\n')

Original Text : 
A father and son are part of a group of seven Florida men accused of prostituting a 16-year-old girl and forcing her to have group sex while they plied her with drugs, prosecutors say. The high school student was reported missing by her school and disappeared for a week while she was driven across Miami-Dade and told to earn $1,288 for the group, according to police. The Miami Herald reports that the girl climbed into a black Ford Taurus with one woman and two men on December 4 after school. Scroll down for video . Accused: Derrick Powell, Sr, (left) and Derrick Powell, Jr, (right) are part of a group charged with trafficking a 16-year-old girl into prostitution while feeding her drugs . She was then given alcohol, marijuana, cocaine and MDMA, an amphetamine-like synthetic drug with hallucinogenic effects. She was coerced into sex acts with at least 16 men over the week she was missing, including two occasions where she was forced to have sex with up to 8 men at once, 

In [4]:
df = pd.DataFrame(columns=['target_text','source_text'])
df['target_text'] = trainning_highlight_ls
df['source_text'] = ['summarize: '+item for item in trainning_article_ls]

In [5]:
df.head()

,target_text,source_text
0,"Bishop John Folda, of North Dakota, is taking ...",summarize: By . Associated Press . PUBLISHED: ...
1,Criminal complaint: Cop used his role to help ...,summarize: (CNN) -- Ralph Mata was an internal...
2,"Craig Eccleston-Todd, 27, had drunk at least t...",summarize: A drunk driver who killed a young w...
3,Nina dos Santos says Europe must be ready to a...,summarize: (CNN) -- With a breezy sweep of his...
4,Fleetwood top of League One after 2-0 win at S...,summarize: Fleetwood are the only team still t...


In [6]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
MAX_EPOCHS = 10

torch.cuda.memory_summary(device=None, abbreviated=False)
torch.utils.checkpoint

model.train(train_df=df[0:(int)(0.7*TRAINNING_SIZE)],
            eval_df=df[(int)(0.7*TRAINNING_SIZE):TRAINNING_SIZE],
            source_max_token_len=MAX_LEN,
            target_max_token_len=SUMMARY_LEN,
            batch_size=4, max_epochs=MAX_EPOCHS, use_gpu=True)

model_path = ''
rootdir = 'outputs/'
for it in os.scandir(rootdir):
    if it.is_dir():
        if 'simplet5-epoch-'+(str)(MAX_EPOCHS-1) in it.path:
            model_path = it.path
            print(model_path)

model.load_model("t5","./"+model_path, use_gpu=True)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

outputs/simplet5-epoch-9-train-loss-0.6147-val-loss-1.9119


In [7]:
no_tune_model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device('cpu')

In [8]:
for index in sample(list(np.arange(len(trainning_article_ls))),5):
    print('Original Text : ')
    print(trainning_article_ls[index])

    Original_article = trainning_article_ls[index]

    print('\n\nSummary Text : ')
    print(trainning_highlight_ls[index])

    Original_Summary = trainning_highlight_ls[index]

    print('\n\nFine tuned Predicted Summary Text : ')
    print(model.predict(trainning_article_ls[index]))

    Predicted_Summary = trainning_article_ls[index]

    rouge = Rouge()
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([Predicted_Summary], Original_Summary)

    print("-----------------")

    print(rouge.get_scores(Predicted_Summary, Original_Summary))

    print("-----------------")

    print(BLEUscore)

    print("-----------------")

    print('\n\nNot Fine tuned Predicted Summary Text : ')
    preprocess_text = trainning_article_ls[index].strip().replace("\n","")
    t5_prepared_Text = "summarize: "+preprocess_text
    tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
    summary_ids = no_tune_model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=30,
                                        max_length=SUMMARY_LEN,
                                        early_stopping=True)
    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(output)
    print('===========================================================================================================\n\n')

Token indices sequence length is longer than the specified maximum sequence length for this model (857 > 512). Running this sequence through the model will result in indexing errors


Original Text : 
By . Hayley O'keeffe . PUBLISHED: . 05:23 EST, 31 October 2013 . | . UPDATED: . 12:53 EST, 31 October 2013 . When artist Alan Dodd was asked by friends to paint a mural on a blank 50-foot-long wall at their home he transformed it into an optical illusion. Friendly faces behind windows and lifelike archways leading to landscapes are painted onto the cottage, in Eye, Suffolk. And the images give an illusion of a make-believe world within. Nationally-renowned artist Alan Dodd stands with the unusual mural that he painted on the side of a friend's home in Eye, Suffolk . The largest painting in the Eye mural is a blacksmith's forge, which takes in the skyline features of the town's church and castle. Quirky characters and also animals feature on other parts of the house side. More... The picture you never thought you'd see... Anna Wintour eating a burger and fries! Artist gives Vogue editor and her fashion friends a calorie-laden makeover . Rare Italian 'San Gennaro' collec

Token indices sequence length is longer than the specified maximum sequence length for this model (859 > 512). Running this sequence through the model will result in indexing errors


["Alan Dodd painted the stunning mural on the side of a friend's home in Eye, Suffolk. He spent 55 days up a ladder to complete the work and says it is a tribute to the town. The artist paints historical scenes and was taught by influential David Hockney."]
-----------------
[{'rouge-1': {'r': 0.8780487804878049, 'p': 0.1094224924012158, 'f': 0.1945945926239591}, 'rouge-2': {'r': 0.44, 'p': 0.04230769230769231, 'f': 0.07719298085564794}, 'rouge-l': {'r': 0.8292682926829268, 'p': 0.1033434650455927, 'f': 0.1837837818131483}}]
-----------------
2.3931422985475966e-05
-----------------


Not Fine tuned Predicted Summary Text : 
artist spent 55 days up a ladder painting the 50-foot-long wall. he says it is an homage to the history of the market town in eye, suffolk - and its history is reflected in the work 'against direct sunlight'


Original Text : 
Jailed: Joshua Sadler, 21, was sentenced to 12 months after admitting dangerous driving . The mother of a teenager who was killed when a fri